In [155]:
# pip install fuzzywuzzy
# pip install python-Levenshtein

In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
import os as os

C:\Users\DELL\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
df_raw=pd.read_csv('inputs/Vocabulearn_all.csv')

#Fonction de filtre globale
def remove_punctuation(text):
    special_punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for ele in text:
        if ele in special_punc:
            text = text.replace(ele, " ")
    return text
langues=list(set(list(df_raw.columns))-set(['ID','ENGLISH_stem','OCCURENCES','INTERVAL']))

def process_global(df):
    df['INTERVAL']=pd.qcut(df['OCCURENCES'],100,duplicates='drop',labels=False)
    df['INTERVAL']=100-df['INTERVAL']
    for column in langues:
        df[column]=df[column].apply(remove_punctuation)
    return df
df_global=process_global(df_raw)

In [7]:
#Split en plusieurs dataset
dicte_langue={}
for langue in langues:
    dicte_langue[langue]=df_global[['ID',langue]]
df_interval=df_global[['ID','INTERVAL']]

In [8]:
langues_bizarre=['CHINESE','KOREAN','JAPAN','RUSSIAN','ARABE','HINDI']
langues_normale=list(set(langues)-set(langues_bizarre))

In [9]:
#Filtre de chaque dataset selon son type
def filtre_langue_normale(df,langue):
    df['_len']=df[langue].apply(len)
    df=df[(df['_len']>3)&(df['_len']<20)]
    df=df.drop(columns=['_len'])
    return df
for langue in langues_normale:
    dicte_langue[langue]=filtre_langue_normale(dicte_langue[langue],langue)

<ipython-input-9-2e837b100cac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_len']=df[langue].apply(len)
<ipython-input-9-2e837b100cac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_len']=df[langue].apply(len)
<ipython-input-9-2e837b100cac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [10]:
langues_bizarre=['CHINESE','KOREAN','JAPAN','RUSSIAN','ARABE','CHINESE']
def filtre_langue_bizarre(df,langue):
    def filter_latin(text):
        text=text.lower()
        alphabet="abcdefghijklmnopqrstvwxyzéèêìîï"
        for letter in alphabet:
            if letter in text:
                return False
        return True
    df['_contain_latin']=df[langue].apply(filter_latin)
    df=df[df['_contain_latin']]
    df=df.drop(columns=['_contain_latin'])
    return df
for langue in langues_bizarre:
    dicte_langue[langue]=filtre_langue_bizarre(dicte_langue[langue],langue)

<ipython-input-10-fd398ee1370f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_contain_latin']=df[langue].apply(filter_latin)


In [188]:
#Filtre post-selection
def merge_and_filter(speak,learn):
    speak_df=dicte_langue[speak]
    learn_df=dicte_langue[learn]
    df=speak_df.merge(learn_df, how='inner', on='ID')
    df=df.merge(df_interval, how='inner', on='ID')
    df['_is_egal']=df[speak]==df[learn]
    df=df[~df['_is_egal']]
    df['_fuzzy_ratio'] = df.apply(lambda x: fuzz.ratio(x[speak], x[learn]), axis=1)
    df=df[df['_fuzzy_ratio']<70]
    df=df.drop(columns=['_is_egal','_fuzzy_ratio'])
    return df

In [216]:
for langue_speak in langues:
    try:
        os.mkdir(path+'\\output\\dataset\\'+langue_speak)
    except:
        print('Error on',langue_speak)
    langues2=[langue for langue in langues if langue!=langue_speak]
    for langue_learn in langues2:
        df=merge_and_filter(langue_speak,langue_learn)
        df=df.sample(frac=1)
        df.to_csv(path+'\\output\\dataset\\'+langue_speak+'\\'+langue_speak+'_'+langue_learn+'.csv',header=True,index=False)
        print(langue_speak,langue_learn,len(df))

PORTUGAIS ENGLISH 5376
PORTUGAIS CHINESE 10923
PORTUGAIS KOREAN 11792
PORTUGAIS JAPAN 10603
PORTUGAIS FRENCH 6601
PORTUGAIS HINDI 12321
PORTUGAIS DEUTSCH 7689
PORTUGAIS SPAIN 4375
PORTUGAIS POLONAIS 7458
PORTUGAIS TURC 7717
PORTUGAIS RUSSIAN 11843
PORTUGAIS ARABE 11495
PORTUGAIS DANOIS 6758
ENGLISH PORTUGAIS 5377
ENGLISH CHINESE 11100
ENGLISH KOREAN 11814
ENGLISH JAPAN 10764
ENGLISH FRENCH 5333
ENGLISH HINDI 12173
ENGLISH DEUTSCH 6365
ENGLISH SPAIN 5741
ENGLISH POLONAIS 7152
ENGLISH TURC 7012
ENGLISH RUSSIAN 11828
ENGLISH ARABE 11452
ENGLISH DANOIS 4852
CHINESE PORTUGAIS 10923
CHINESE ENGLISH 11100
CHINESE KOREAN 11523
CHINESE JAPAN 9826
CHINESE FRENCH 10664
CHINESE HINDI 11894
CHINESE DEUTSCH 10750
CHINESE SPAIN 10641
CHINESE POLONAIS 10647
CHINESE TURC 10700
CHINESE RUSSIAN 11524
CHINESE ARABE 11242
CHINESE DANOIS 10942
KOREAN PORTUGAIS 11792
KOREAN ENGLISH 11814
KOREAN CHINESE 11523
KOREAN JAPAN 11002
KOREAN FRENCH 11423
KOREAN HINDI 12870
KOREAN DEUTSCH 11574
KOREAN SPAIN 11412
KOR